In [1]:
import pandas as pd
from pathlib import Path
from nbgrader.api import Gradebook

In [2]:
COURSE_ID = "itmo_recsys_2025_spring"
DB_URL = "sqlite:////app/gradebook.db"

In [3]:
gradebook = Gradebook(db_url=DB_URL, course_id=COURSE_ID)

In [9]:
gradebook.find_submission("hw_2", "nbelyaev1")

SubmittedAssignment<hw_2 for nbelyaev1>

In [ ]:
gradebook.

In [5]:
ass_hw2 = gradebook.assignments[0]

In [8]:
ass_hw2

Assignment<hw_2>

In [7]:
ass_hw2.notebooks

[Notebook<hw_2/metrics>]